In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings('ignore')

#to display all rows columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

In [20]:
df = pd.read_csv('all_matches.csv')

In [21]:
df['match_id'].nunique()

1095

In [22]:
df.shape

(260920, 22)

In [23]:
# Removing all the matches since 2023 final including the final

df = df[df['match_id']< 1370353]

In [24]:
df['match_id'].nunique()

1023

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128290 entries, 0 to 128289
Data columns (total 22 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   match_id                128290 non-null  int64  
 1   season                  128289 non-null  object 
 2   start_date              128289 non-null  object 
 3   venue                   128289 non-null  object 
 4   innings                 128289 non-null  float64
 5   ball                    128289 non-null  float64
 6   batting_team            128289 non-null  object 
 7   bowling_team            128289 non-null  object 
 8   striker                 128289 non-null  object 
 9   non_striker             128289 non-null  object 
 10  bowler                  128289 non-null  object 
 11  runs_off_bat            128289 non-null  float64
 12  extras                  128289 non-null  float64
 13  wides                   3912 non-null    float64
 14  noballs             

In [25]:
df.batting_team.unique()

array(['Kolkata Knight Riders', 'Royal Challengers Bangalore',
       'Chennai Super Kings', 'Kings XI Punjab', 'Rajasthan Royals',
       'Delhi Daredevils', 'Deccan Chargers', 'Mumbai Indians',
       'Kochi Tuskers Kerala', 'Pune Warriors', 'Sunrisers Hyderabad',
       'Rising Pune Supergiants', 'Gujarat Lions',
       'Rising Pune Supergiant', 'Delhi Capitals', 'Punjab Kings',
       'Lucknow Super Giants', 'Gujarat Titans'], dtype=object)

In [26]:
t1 = 'Gujarat Titans'
t2 = 'Chennai Super Kings'

In [27]:
df.innings.value_counts()

,count
innings,
1,126003
2,117437
3,77
4,72
5,8
6,4


In [28]:
df = df[(df.innings == 1) | (df.innings == 2)]

In [29]:
df.innings.value_counts()

,count
innings,
1,126003
2,117437


In [30]:
df['total_runs'] = df['runs_off_bat'] + df['extras']

In [31]:
df['isOut'] = df['player_dismissed'].apply(lambda x: 1 if type(x) == type('str') else 0)

In [32]:
#1370353 is the match id
outcomes = [0, 1, 2, 3, 4, 6, 'w']

In [33]:
df[df.batting_team==t2]['total_runs'].value_counts()

,count
total_runs,
1,11291
0,9153
4,3018
2,1827
6,1382
3,106
5,47
7,9


In [35]:
df[df.batting_team==t2].isOut.sum()

1161

In [36]:
frequencies = [9153, 11291,1827,106, 3018, 1382, 1161 ]

In [37]:
percentages = [x/sum(frequencies) for x in frequencies]

In [38]:
percentages

[0.3276182976591023,
 0.404144892261436,
 0.06539480277757892,
 0.003794115541556303,
 0.10802491230582004,
 0.049466676211611424,
 0.041556303242894985]

In [39]:
cumulative = np.cumsum(percentages)

In [40]:
cumulative

array([0.3276183 , 0.73176319, 0.79715799, 0.80095211, 0.90897702,
       0.9584437 , 1.        ])

In [43]:
def predict_runs(target, current_score, current_wickets, current_overs, total_balls):

    # pb values of both teams

    pred_runs = current_score
    pred_wks = current_wickets
    leftover_balls = total_balls - current_overs*6

    for i in range(leftover_balls):
        r_value = np.random.random()

        if r_value <= cumulative[0]:
            pred_runs += 0
        elif r_value <= cumulative[1]:
            pred_runs += 1
        elif r_value <= cumulative[2]:
            pred_runs += 2
        elif r_value <= cumulative[3]:
            pred_runs += 3
        elif r_value <= cumulative[4]:
            pred_runs += 4
        elif r_value <= cumulative[5]:
            pred_runs += 6
        else:
            pred_runs += 0
            pred_wks += 1
            if pred_wks == 10:
                break
        if pred_runs > target:
            break
        # print('pred_runs: ', pred_runs)
        # print('pred_wks: ', pred_wks)

    return pred_runs

In [46]:
predict_runs(171,0,0,0,90 )

109

In [47]:
def get_win(pred_runs, target):
    if pred_runs > target:
        return 'win'
    elif pred_runs == target:
        return 'tie'
    else:
        return 'lose'

In [61]:
target = 171
total_balls = 90

current_score = 150#100
current_wickets = 5#2
current_overs = 9#9

iter_count = 100

runs_ls = []
results_ls = []

win_count = 0
tie_count = 0
lose_count = 0

for i in range(iter_count):
    pred_runs = predict_runs(target, current_score, current_wickets, current_overs, total_balls)
    runs_ls.append(pred_runs)
    print(pred_runs)
    result_pred = get_win(pred_runs, target)
    results_ls.append(result_pred)

    if result_pred == 'win':
        win_count += 1
    elif result_pred == 'tie':
        tie_count += 1
    else:
        lose_count +=1

172
172
175
173
172
172
172
172
172
174
172
173
173
173
173
172
174
175
175
176
173
172
174
172
175
173
176
176
175
173
173
172
174
177
174
174
172
176
172
175
175
174
173
172
172
174
173
173
173
173
172
174
172
176
172
172
172
172
172
176
175
172
172
173
176
175
174
174
173
175
174
174
174
173
172
172
172
172
174
175
176
174
176
174
175
174
174
173
172
172
172
172
173
173
172
172
172
175
172
174


In [62]:
win_count, tie_count, lose_count

(100, 0, 0)

In [59]:
def find_runs(current_score, target, current_wickets, at_overs, total_balls):
    runs_ls = []
    results_ls = []

    req_runs = []
    win_ls = []

    for i in range(current_score, target + 1):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
            pred_runs = predict_runs(target, i, current_wickets,
                                     at_overs, total_balls)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

            win_ls.append(win_count)
            req_runs.append(i)
            # print('runs: ', i, ' win%: ', win_count)

    required_runs = current_score
    for i in range(len(req_runs)):
        if win_ls[i] >= 50:
            required_runs = req_runs[i]
            # print('Runs to be: ', req_runs[i])
            break

    return required_runs


In [63]:
find_runs(100, 171, 3, 9, 90)

126

In [65]:
def find_wickets(current_score, target, current_wickets, at_overs,total_balls):

#     find_runs(current_score, target, current_wickets, at_overs)
    req_runs = find_runs(current_score, target, current_wickets, at_overs, total_balls)

    runs_ls = []
    results_ls = []

    req_wks = []
    win_ls = []

    for i in range(current_wickets, 10):
        win_count = 0
        tie_count = 0
        lose_count = 0

        for j in range(100):
#             pred_runs = predict_runs(target, req_runs, i, at_overs)
            pred_runs = predict_runs(target, current_score, i, at_overs,total_balls)
            runs_ls.append(pred_runs)
            result_pred = get_win(pred_runs, target)
            results_ls.append(result_pred)

            if result_pred == 'win':
                win_count += 1
            elif result_pred == 'tie':
                tie_count += 1
            else:
                lose_count +=1

        win_ls.append(win_count)
        req_wks.append(i)
#         print('wickets: ', i, ' win%: ', win_count)

    req_wicket_value = current_wickets

    for i in range(len(req_wks)):
        if (win_ls[i] < 45)  :
            req_wicket_value = req_wks[i]
            break

    return req_wicket_value

In [67]:
find_wickets(126, 171, 3, 9, 90)

5

In [74]:
%matplotlib inline
from ipywidgets import interactive
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np

def find_runs_wickets(current_wks, at_overs, target_score):
    plt.figure(figsize = (16, 6))
    # x = np.linspace(-10, 10, num=1000)
    total_balls = 90
    x = np.array(list(range(21)))
    req_value = find_runs(100, target_score, current_wks, at_overs, total_balls)
    req_wk_value = find_wickets(100, target_score, current_wks, at_overs, total_balls)

    if at_overs == 9:
        req_value = 100
        req_wk_value = 3


    # print(req_value)
    y = np.array([req_value for i in range(21)])

    # plt.plot(x, current_overs * x + target_score)
    # plt.plot(x, y)
    plt.scatter(at_overs, req_value, s = 1200, color = 'red')
    plt.axhline(target_score, ls = '--', color = 'blue')
    plt.text( 1, target_score + 10, 'Target Score :' + str(target_score) , color = 'darkblue', fontsize = 13)
    plt.text( at_overs, req_value, str(req_value) + '/' + str(req_wk_value), color = 'white', fontsize = 12,  horizontalalignment='center', verticalalignment='center')
    plt.text(at_overs, req_value - 30, 'CSK has to be at ' + str(req_value) + '/' +  str(req_wk_value) + ' after ' + str(at_overs) + ' ov', horizontalalignment='center')
    plt.ylim(50, target_score + 50)
    plt.xticks(x)
    plt.title('Where should CSK be?', fontsize = 20)
    plt.xlabel('Overs')
    plt.ylabel('Score')
    plt.show()

# x=widgets.IntSlider(min=-10, max=30, step=1, value=10)

# find_wickets(current_score, target, current_wickets, at_overs)
# find_wickets(87, 167, 1, 14)


In [75]:
print('current_score = CSK: 100/3 (9 overs)')
print('Game Shortened to 15 Overs')

interactive_plot = interactive(find_runs_wickets, current_wks = widgets.IntSlider(min=1, max=10, step=1, value=3),  at_overs=widgets.IntSlider(min=10, max=15, step=1, value=9), target_score = widgets.IntSlider(min=0, max=250, step=1, value=171)
                               )
output = interactive_plot.children[-1]
output.layout.height = '450px'
interactive_plot

current_score = CSK: 100/3 (9 overs)
Game Shortened to 15 Overs


interactive(children=(IntSlider(value=3, description='current_wks', max=10, min=1), IntSlider(value=10, descri…